# Scrap 

just for building matrices 

In [96]:
import pandas as pd
import os 
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import time

In [2]:
%reload_ext autoreload
%autoreload 2

# Tell python where to look for modules. 
# Depending on how your jupyter handles working directories, this may not be needed.
import sys
sys.path.append('../../hourly-egrid/')

from src.new_consumed import *
import src.consumed as old_consumed

2022-07-28 14:20:36,201 - pudl.workspace.setup - INFO - PUDL user settings file .pudl.yml not found.
2022-07-28 14:20:36,202 - pudl.workspace.setup - INFO - PUDL user settings file .pudl.yml not found.


In [3]:
from gridemissions.load import BaData
from gridemissions.eia_api import KEYS, SRC

In [4]:
eia930 = BaData("../data/outputs/2020/eia930/eia930_elec.csv")

In [5]:
default_factors = get_average_emission_factors()

In [6]:
# Load all rates 
#def load_power_sector(prefix="2020/"):
prefix = "2020/"
rates = {}
gens = {}
for f in os.listdir(f"../data/results/{prefix}/power_sector_data/hourly/us_units/"):
    if ".DS_Store" in f:
        continue 
    this_ba = pd.read_csv(f"../data/results/{prefix}/power_sector_data/hourly/us_units/"+f, index_col="datetime_utc",parse_dates=True)
    this_ba = this_ba[this_ba.fuel_category=="total"]
    ba_name = f.replace(".csv", "")
    for adj in ADJUSTMENTS:
        for pol in POLLS:
            this_rate = rates.get((adj, pol), {})
            this_rate[ba_name] = this_ba[get_column(pol, adjustment=adj)]
            rates[(adj,pol)] = this_rate
    gens[ba_name] = this_ba["net_generation_mwh"]



#eia930.regions

In [7]:
generation = pd.DataFrame(data=gens)

In [8]:
# Identify shared BAs 
regions = set(eia930.regions)
regions = regions.intersection(set(generation.columns))
# Add back import-only regions 
regions = regions.union(set(IMPORT_REGIONS))
regions = list(regions)

In [9]:
# build result DF per output file
results = {}
cols = []
for pol in POLLS:
    for adj in ADJUSTMENTS: 
        cols.append(get_rate_column(pol, adjustment=adj, generated=False))
        cols.append(get_column(pol, adjustment=adj))
cols.append("consumed_mwh")
for ba in regions:
    results[ba] = pd.DataFrame(index=generation.index, columns=cols)

In [134]:
class HourlyConsumed():
    def __init__(self, eia930_file:str, prefix:str, year:int):
        self.prefix=prefix
        self.year=year

        # 930 data
        self.eia930 = BaData(eia930_file)

        # Emission factors for non-US bas
        self.default_factors = get_average_emission_factors()

        # Load generated rates, save to self.generated
        self.rates, self.generation = self._load_rates()

        # Build result df 
        self.results = self._build_results()

        # Identify shared BAs 
        regions = set(eia930.regions)
        regions = regions.intersection(set(self.generation.columns))
        # Add back import-only regions 
        regions = regions.union(set(IMPORT_REGIONS))
        self.regions = list(regions)

    def _build_results(self):
        # build result DF per output file
        results = {}
        cols = []
        for pol in POLLS:
            for adj in ADJUSTMENTS: 
                cols.append(get_rate_column(pol, adjustment=adj, generated=False))
                cols.append(get_column(pol, adjustment=adj))
        cols.append("consumed_mwh")
        for ba in regions:
            results[ba] = pd.DataFrame(index=generation.index, columns=cols)
        return results
    

    def _load_rates(self):
        # Load all rates 
        rates = {}
        gens = {}
        for f in os.listdir(f"../data/results/{self.prefix}/power_sector_data/hourly/us_units/"):
            if ".DS_Store" in f:
                continue 
            this_ba = pd.read_csv(f"../data/results/{self.prefix}/power_sector_data/hourly/us_units/"+f, index_col="datetime_utc",parse_dates=True)
            this_ba = this_ba[this_ba.fuel_category=="total"]
            ba_name = f.replace(".csv", "")
            for adj in ADJUSTMENTS:
                for pol in POLLS:
                    this_rate = rates.get((adj, pol), {})
                    this_rate[ba_name] = this_ba[get_column(pol, adjustment=adj)]
                    rates[(adj,pol)] = this_rate
            gens[ba_name] = this_ba["net_generation_mwh"]

        # Make each rate into a DF and add emissions for import-only regions 
        for pol in POLLS:
            for adj in ADJUSTMENTS:
                # Calculate emissions 
                emissions = pd.DataFrame(rates[(adj, pol)])

                # Add import regions to emissions DF
                for ba in IMPORT_REGIONS:
                    gen_cols = [(src, KEYS["E"]["SRC_%s" % src]%ba) for src in SRC]
                    gen_cols = [(src, col) for src, col in gen_cols if col in eia930.df.columns]
                    emissions.loc[:,ba] = eia930.df.apply(
                        lambda x: sum(default_factors[pol][adj][src] * x[col] for src, col in gen_cols), axis=1
                    )

                rates[((adj, pol))] = emissions

        return rates, pd.DataFrame(data=gens)

    def build_matrices(self, pol:str, adj:str, date):
        # return emissions, interchange, and generation
        # Build transmission matrix from cleaned 
        ID = np.zeros((len(self.regions), len(self.regions)))
        for i, ri in enumerate(self.regions):
            for j, rj in enumerate(self.regions):
                if KEYS["E"]["ID"] % (ri, rj) in self.eia930.df.columns:
                    ID[i][j] = self.eia930.df.loc[date,KEYS["E"]["ID"] % (ri, rj)]

        # Build emission array, using default per-fuel factors for import-only regions (above)
        E = self.rates[(adj, pol)].loc[date, self.regions].to_numpy()

        # Build generation array, using 930 for import-only regions
        G = np.zeros(len(self.regions))
        for (i,r) in enumerate(self.regions): 
            if r in IMPORT_REGIONS:
                G[i] = self.eia930.df.loc[date,KEYS["E"]["NG"]%r]
            else:
                G[i] = self.generation.loc[date,r]

        # In some cases, we have zero generation but non-zero transmission 
        # usually due to imputed zeros during physics-based cleaning being set to 1.0
        # but sometimes due to ok values being set to 1.0ß
        to_fix = (ID.sum(axis=1) > 0) & (G==0)
        ID[:,to_fix] = 0
        ID[to_fix,:] = 0

        return E,G,ID


    def run(self):
        for pol in POLLS:
            for adj in ADJUSTMENTS:
                col = get_column(pol, adjustment=adj)
                print(f"{pol}, {adj}", end="...")
                # Calculate emissions 

                for date in generation.index:
                    #ref = time.perf_counter()
                    E, G, ID = self.build_matrices(pol, adj, date)
                    #print(f"{time.perf_counter() - ref} \tfor building matrices")
                    # If we have NaNs, don't bother
                    if (np.isnan(G).sum() > 0) or (np.isnan(E).sum() > 0) or (np.isnan(ID).sum() > 0):
                        consumed_emissions = np.full(len(regions), np.nan)
                    else: 
                        # Run
                        try:
                            #ref = time.perf_counter()
                            consumed_emissions, _ = consumption_emissions(E,G,ID)
                            #print(f"{time.perf_counter() - ref} \tfor running")
                        except:
                            print(f"Warning: singular matrix on {date}")
                            consumed_emissions = np.full(len(regions), np.nan)

                    # Export 
                    # TODO: this may be slow, coud make faster using intermediate output with friendlier format
                    #ref = time.perf_counter()
                    for (i,r) in enumerate(self.regions): 
                        self.results[r].loc[date,col] = consumed_emissions[i]
                    #print(f"{time.perf_counter() - ref} \tfor saving")

            

        

In [135]:
hc = HourlyConsumed("../data/outputs/2020/eia930/eia930_elec.csv", "/2020/", 2020)

In [139]:
hc.results["ISNE"].head(20)

,consumed_co2_rate_lb_per_mwh_for_electricity,co2_mass_lb_for_electricity,consumed_co2_rate_lb_per_mwh_for_electricity_adjusted,co2_mass_lb_for_electricity_adjusted,consumed_ch4_rate_lb_per_mwh_for_electricity,ch4_mass_lb_for_electricity,consumed_ch4_rate_lb_per_mwh_for_electricity_adjusted,ch4_mass_lb_for_electricity_adjusted,consumed_n2o_rate_lb_per_mwh_for_electricity,n2o_mass_lb_for_electricity,consumed_n2o_rate_lb_per_mwh_for_electricity_adjusted,n2o_mass_lb_for_electricity_adjusted,consumed_co2e_rate_lb_per_mwh_for_electricity,co2e_mass_lb_for_electricity,consumed_co2e_rate_lb_per_mwh_for_electricity_adjusted,co2e_mass_lb_for_electricity_adjusted,consumed_nox_rate_lb_per_mwh_for_electricity,nox_mass_lb_for_electricity,consumed_nox_rate_lb_per_mwh_for_electricity_adjusted,nox_mass_lb_for_electricity_adjusted,consumed_so2_rate_lb_per_mwh_for_electricity,so2_mass_lb_for_electricity,consumed_so2_rate_lb_per_mwh_for_electricity_adjusted,so2_mass_lb_for_electricity_adjusted,consumed_mwh
datetime_utc,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 06:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 08:00:00+00:00,NaN,516.116939,NaN,392.717271,NaN,0.071112,NaN,0.071017,NaN,0.009321,NaN,0.009312,NaN,521.284264,NaN,397.878624,NaN,0.449586,NaN,0.446434,NaN,0.114913,NaN,0.113098,NaN
2020-01-01 09:00:00+00:00,NaN,514.751772,NaN,392.326469,NaN,0.0706,NaN,0.070506,NaN,0.009253,NaN,0.009244,NaN,519.881172,NaN,397.449959,NaN,0.453115,NaN,0.449996,NaN,0.114328,NaN,0.112531,NaN
2020-01-01 10:00:00+00:00,NaN,515.107565,NaN,394.904994,NaN,0.069528,NaN,0.069434,NaN,0.009106,NaN,0.009097,NaN,520.157394,NaN,399.948873,NaN,0.450374,NaN,0.447235,NaN,0.112551,NaN,0.110742,NaN
2020-01-01 11:00:00+00:00,NaN,519.524225,NaN,401.437505,NaN,0.068515,NaN,0.068422,NaN,0.008964,NaN,0.008954,NaN,524.498373,NaN,406.40583,NaN,0.442959,NaN,0.439887,NaN,0.110417,NaN,0.108647,NaN
2020-01-01 12:00:00+00:00,NaN,507.873127,NaN,394.82366,NaN,0.066545,NaN,0.066455,NaN,0.008704,NaN,0.008695,NaN,512.702653,NaN,399.647488,NaN,0.441099,NaN,0.438092,NaN,0.108468,NaN,0.106736,NaN


In [136]:
hc.run()

CO2, for_electricity...CO2, for_electricity_adjusted...CH4, for_electricity...CH4, for_electricity_adjusted...N2O, for_electricity...N2O, for_electricity_adjusted...CO2E, for_electricity...CO2E, for_electricity_adjusted...NOX, for_electricity...NOX, for_electricity_adjusted...SO2, for_electricity...SO2, for_electricity_adjusted...

In [104]:
problem_date = "2020-01-01T04:00:00+00:00"
ok_date = "2020-01-02T03:00:00+00:00"

In [128]:
np.isnan(ID).sum()

0

In [105]:
E, G, ID = hc.build_matrices("CO2", "for_electricity", problem_date)

In [106]:
E_ok, G_ok, ID_ok = hc.build_matrices("CO2", "for_electricity", ok_date)
ok = consumption_emissions(E_ok, G_ok, ID_ok)

In [107]:
consumption_emissions(E,G,ID)

LinAlgError: SVD did not converge

In [108]:
G

array([           nan,            nan,            nan,            nan,
                  nan, 1.05416487e+03,            nan,            nan,
       2.02095585e+03,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
       4.09533945e+02,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan, 1.91009484e+03, 1.94752102e+02,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
      

In [49]:
eia_orig = BaData("../data/outputs/2020/eia930/eia930_rolling.csv")

In [43]:
regions = np.array(hc.regions)
print(regions[G==0])
print(regions[G_ok==0])

['WWA' 'CPLW' 'HGMA' 'NSB' 'HST']
['CPLW' 'HGMA' 'NSB' 'HST']


In [87]:
to_fix = (ID.sum(axis=1) > 0) & (G==0)
new_ID = ID.copy()
new_ID[:,to_fix] = 0
new_ID[to_fix,:] = 0


In [88]:
consumption_emissions(E, G, new_ID)

(array([ 5.59833598e+02,  1.22498119e+03,  4.96157213e+02,  8.10415105e+02,
         4.88138798e+02,  3.89687499e-02,  1.19061238e+03,  1.24592078e+03,
         8.48306632e+02, -1.39237844e-12,  2.82260761e+02,  1.16410125e+02,
         5.72754616e+02,  8.46623119e+02,  2.26312011e+02,  5.95992845e+02,
         0.00000000e+00,  8.67264114e+02,  8.82173142e+02,  1.75527559e+03,
         9.79844110e+02,  1.39880193e+03,  1.46094106e+03,  5.72319447e+02,
         4.88138798e+02,  3.43518296e+02,  4.38408386e+02,  4.75158720e+02,
         4.37483839e+02,  1.36421347e+01,  4.08988197e+02,  4.99407714e+02,
         8.48306632e+02,  4.01815874e+02,  0.00000000e+00,  7.51996578e+02,
         2.22023392e+03,  5.34033487e+02,  1.14669984e+03,  6.22157400e+02,
         8.82425542e+02,  1.38426928e+02,  6.22157400e+02,  1.96628734e+03,
         8.73541902e+02,  8.26156548e+02,  3.89687499e-02,  2.77207340e+02,
         1.53018430e+03,  7.89151913e+02,  1.82415844e+03,  4.99407714e+02,
         8.7

In [56]:
wwa_i = hc.regions.index("WWA")
new_ID = ID.copy()
new_ID[:,wwa_i] = 0
new_ID[wwa_i, :] = 0

In [57]:
consumption_emissions(E,G,new_ID)

(array([ 5.59833598e+02,  1.22498119e+03,  4.96157213e+02,  8.10415105e+02,
         4.88138798e+02,  3.89687499e-02,  1.19061238e+03,  1.24592078e+03,
         8.48306632e+02, -1.39237844e-12,  2.82260761e+02,  1.16410125e+02,
         5.72754616e+02,  8.46623119e+02,  2.26312011e+02,  5.95992845e+02,
         0.00000000e+00,  8.67264114e+02,  8.82173142e+02,  1.75527559e+03,
         9.79844110e+02,  1.39880193e+03,  1.46094106e+03,  5.72319447e+02,
         4.88138798e+02,  3.43518296e+02,  4.38408386e+02,  4.75158720e+02,
         4.37483839e+02,  1.36421347e+01,  4.08988197e+02,  4.99407714e+02,
         8.48306632e+02,  4.01815874e+02,  0.00000000e+00,  7.51996578e+02,
         2.22023392e+03,  5.34033487e+02,  1.14669984e+03,  6.22157400e+02,
         8.82425542e+02,  1.38426928e+02,  6.22157400e+02,  1.96628734e+03,
         8.73541902e+02,  8.26156548e+02,  3.89687499e-02,  2.77207340e+02,
         1.53018430e+03,  7.89151913e+02,  1.82415844e+03,  4.99407714e+02,
         8.7

In [75]:

print(eia_orig.df.loc[ok_date, "EBA.WWA-ALL.NG.H"])
print(hc.eia930.df.loc[ok_date, "EBA.WWA-ALL.NG.H"])

print("problem date")
print(eia_orig.df.loc[problem_date, "EBA.WWA-ALL.TI.H"])
print(eia_orig.df.loc[problem_date, "EBA.WWA-ALL.NG.H"])
print(hc.eia930.df.loc[problem_date, "EBA.WWA-ALL.NG.H"])

72.0
74.76934034670522
problem date
6.0
6.0
8.999999999999995


In [45]:
hc.eia930.df.columns

Index(['EBA.AEC-ALL.NG.H', 'EBA.AECI-ALL.NG.H', 'EBA.AESO-ALL.NG.H', 'EBA.AVA-ALL.NG.H', 'EBA.AVRN-ALL.NG.H', 'EBA.AZPS-ALL.NG.H', 'EBA.BANC-ALL.NG.H', 'EBA.BCHA-ALL.NG.H', 'EBA.BPAT-ALL.NG.H', 'EBA.CEN-ALL.NG.H',
       ...
       'EBA.TEC-ALL.NG.WND.H', 'EBA.TEPC-ALL.NG.WND.H', 'EBA.TIDC-ALL.NG.WND.H', 'EBA.TPWR-ALL.NG.WND.H', 'EBA.TVA-ALL.NG.WND.H', 'EBA.WACM-ALL.NG.WND.H', 'EBA.WALC-ALL.NG.WND.H', 'EBA.WAUW-ALL.NG.WND.H', 'EBA.WWA-ALL.NG.WND.H', 'EBA.YAD-ALL.NG.WND.H'], dtype='object', length=1150)

In [37]:
px.scatter(x=hc.regions, y=[G, G_ok])

## what's up with WWA?

In [59]:
plant_meta = pd.read_csv("../data/results/2020/plant_data/plant_static_attributes.csv", index_col="plant_id_eia")

In [61]:
plant_meta[plant_meta.ba_code == "WWA"]

,Unnamed: 0,plant_primary_fuel,data_availability,ba_code,ba_code_physical,state,distribution_flag,fuel_category,fuel_category_eia930,timezone
plant_id_eia,,,,,,,,,,
57995,5631,WND,eia_only,WWA,WWA,MT,False,wind,wind,America/Denver
920912,345,NaN,NaN,WWA,NaN,NaN,NaN,wind,NaN,NaN


In [62]:
plant_gen = pd.read_csv("../data/results/2020/plant_data/hourly/us_units/CEMS_plant_data.csv")
syn_plant_gen = pd.read_csv("../data/results/2020/plant_data/hourly/us_units/synthetic_plant_data.csv")

In [63]:
p1 = plant_gen[plant_gen.plant_id_eia==57995]
ps = syn_plant_gen[syn_plant_gen.plant_id_eia==920912]

In [77]:
fig = px.line(ps, x="datetime_utc", y="net_generation_mwh")

fig.add_trace(go.Scatter(x=hc.eia930.df.index, y=hc.eia930.df["EBA.WWA-ALL.NG.H"],
                    mode='lines',
                    name='EIA 930 net gen'))

fig.add_trace(go.Scatter(x=hc.eia930.df.index, y=hc.eia930.df["EBA.WWA-ALL.TI.H"],
                    mode='lines',
                    name='EIA 930 total interchange'))

fig.add_trace(go.Scatter(x=eia_orig.df.index, y=eia_orig.df["EBA.WWA-ALL.NG.H"],
                    mode='lines',
                    name='EIA 930 original generation (before physics-based cleaning)'))

fig.add_trace(go.Scatter(x=eia_orig.df.index, y=eia_orig.df["EBA.WWA-ALL.TI.H"],
                    mode='lines',
                    name='EIA 930 original transmission (before physics-based cleaning)'))


fig.show()

In [65]:
profiles = 

,plant_id_eia,datetime_utc,report_date,net_generation_mwh,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,co2_mass_lb,ch4_mass_lb,n2o_mass_lb,co2e_mass_lb,nox_mass_lb,so2_mass_lb,co2_mass_lb_for_electricity,ch4_mass_lb_for_electricity,n2o_mass_lb_for_electricity,co2e_mass_lb_for_electricity,nox_mass_lb_for_electricity,so2_mass_lb_for_electricity,co2_mass_lb_adjusted,ch4_mass_lb_adjusted,n2o_mass_lb_adjusted,co2e_mass_lb_adjusted,nox_mass_lb_adjusted,so2_mass_lb_adjusted,co2_mass_lb_for_electricity_adjusted,ch4_mass_lb_for_electricity_adjusted,n2o_mass_lb_for_electricity_adjusted,co2e_mass_lb_for_electricity_adjusted,nox_mass_lb_for_electricity_adjusted,so2_mass_lb_for_electricity_adjusted
0,3,2020-01-01 06:00:00+00:00,2020-01-01,972.17,8342.3,8342.3,1377800.0,116.99,16.64,1386736.76,382.4,70.42,1377800.0,116.99,16.64,1386736.76,382.4,70.42,1377800.0,116.99,16.64,1386736.76,382.4,70.42,1377800.0,116.99,16.64,1386736.76,382.4,70.42
1,3,2020-01-01 07:00:00+00:00,2020-01-01,972.17,8352.5,8352.5,1380400.0,117.40,16.70,1389368.57,401.6,82.02,1380400.0,117.40,16.70,1389368.57,401.6,82.02,1380400.0,117.40,16.70,1389368.57,401.6,82.02,1380400.0,117.40,16.70,1389368.57,401.6,82.02
2,3,2020-01-01 08:00:00+00:00,2020-01-01,969.23,8354.4,8354.4,1382000.0,117.71,16.75,1390993.14,474.8,54.91,1382000.0,117.71,16.75,1390993.14,474.8,54.91,1382000.0,117.71,16.75,1390993.14,474.8,54.91,1382000.0,117.71,16.75,1390993.14,474.8,54.91
3,3,2020-01-01 09:00:00+00:00,2020-01-01,967.27,8317.9,8317.9,1375400.0,117.04,16.65,1384341.84,463.2,44.40,1375400.0,117.04,16.65,1384341.84,463.2,44.40,1375400.0,117.04,16.65,1384341.84,463.2,44.40,1375400.0,117.04,16.65,1384341.84,463.2,44.40
4,3,2020-01-01 10:00:00+00:00,2020-01-01,984.20,8526.9,8526.9,1409600.0,119.94,17.07,1418763.53,474.6,54.06,1409600.0,119.94,17.07,1418763.53,474.6,54.06,1409600.0,119.94,17.07,1418763.53,474.6,54.06,1409600.0,119.94,17.07,1418763.53,474.6,54.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10110508,61242,2021-01-01 01:00:00+00:00,2020-12-01,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
10110509,61242,2021-01-01 02:00:00+00:00,2020-12-01,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
10110510,61242,2021-01-01 03:00:00+00:00,2020-12-01,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
10110511,61242,2021-01-01 04:00:00+00:00,2020-12-01,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00
